In [229]:
import pandas as pd

df_X = pd.read_csv('X_NorthAmerica.csv')
df_X


,CompNo,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,ni2tatrend,sizelevel,sizetrend,m2b,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin
0,15984000.0,1999.0,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,NaN,1.027931,0.000000,NaN,NaN,0.0,0.0,0.000000,2.324205,0
1,15984000.0,1999.0,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,NaN,1.351854,0.323923,NaN,NaN,0.0,0.0,0.000000,2.373554,0
2,15984000.0,1999.0,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,NaN,1.358506,0.013304,NaN,0.274280,0.0,0.0,0.000000,2.401488,0
3,15984000.0,1999.0,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,0.0,1.362518,0.012036,5.082778,0.276896,0.0,0.0,0.000000,2.455890,0
4,15984000.0,1999.0,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,0.0,1.336588,-0.103719,4.504762,0.259755,0.0,0.0,0.000000,2.455596,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753072,212037000.0,2023.0,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,NaN,-2.751389,0.000000,NaN,NaN,0.0,0.0,0.000000,2.087496,0
2753073,212083000.0,2023.0,6,0.038825,-0.010290,NaN,NaN,0.000000,0.0,NaN,NaN,0.048190,0.000000,NaN,NaN,NaN,NaN,3.405230,0.000000,1
2753074,212083000.0,2023.0,7,0.015960,-0.010290,NaN,NaN,0.000000,0.0,NaN,NaN,-0.675007,-0.723197,NaN,NaN,NaN,NaN,3.453191,0.000000,1
2753075,212122000.0,2023.0,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,NaN,-2.258285,0.000000,NaN,NaN,0.0,0.0,0.000000,2.087496,0


In [230]:
## preprocess the X dataset

df_X['yyyy'] = df_X['yyyy'].astype(int)
df_X['mm'] = df_X['mm'].astype(str)

for index, row in df_X.iterrows():
    if len(row['mm']) == 1:  
        df_X.at[index, 'EventDate'] = str(row['yyyy']) + '0' + row['mm']
    else:
        df_X.at[index, 'EventDate'] = str(row['yyyy']) + row['mm']

    df_X.at[index, 'CompNo'] = row['CompNo']/1000
df_X['CompNo'] = df_X['CompNo'].astype(int)

# Convert the "EventDate" column to datetime format
df_X['EventDate'] = pd.to_datetime(df_X['EventDate'], format='%Y%m')

# # Format the "EventDate" column to show year and month (yyyy mm)
df_X['EventDate_string'] = df_X['EventDate'].dt.strftime('%Y %m')

# Rename the "CompNo" column to "CompanyNumber"
df_X = df_X.rename(columns={'CompNo': 'CompanyNumber'})
df_X.head()

C:\Users\shiho\AppData\Local\Temp\ipykernel_54368\2975454212.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '199903' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_X.at[index, 'EventDate'] = str(row['yyyy']) + '0' + row['mm']


,CompanyNumber,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,sizetrend,m2b,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,EventDate,EventDate_string
0,15984,1999,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,0.0,0.0,0.0,2.324205,0,1999-03-01,1999 03
1,15984,1999,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,...,0.323923,NaN,NaN,0.0,0.0,0.0,2.373554,0,1999-04-01,1999 04
2,15984,1999,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,...,0.013304,NaN,0.274280,0.0,0.0,0.0,2.401488,0,1999-05-01,1999 05
3,15984,1999,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,...,0.012036,5.082778,0.276896,0.0,0.0,0.0,2.455890,0,1999-06-01,1999 06
4,15984,1999,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,...,-0.103719,4.504762,0.259755,0.0,0.0,0.0,2.455596,0,1999-07-01,1999 07


In [241]:
import pandas as pd
df_Y = pd.read_csv('Y_NorthAmerica.csv')

df_Y['EventDate'] = pd.to_datetime(df_Y['EventDate'], format='%Y%m%d')
df_Y['Time_Begin'] = pd.to_datetime(df_Y['Time_Begin'], format='%Y%m%d')

# # Format the "EventDate" column to show year and month (yyyy mm)
df_Y['EventDate_string'] = df_Y['EventDate'].dt.strftime('%Y %m')
# # Format the "EventDate" column to show year and month (yyyy mm)
# df_Y['EventDate'] = df_Y['EventDate'].dt.strftime('%Y %m')

# # Format the "EventDate" column to show year and month (yyyy mm)
# df_Y['Time_Begin'] = df_Y['Time_Begin'].dt.strftime('%Y %m')

df_Y = df_Y.rename(columns={'CompNo': 'CompanyNumber', 'Event_type': 'EventType','Time_Begin': 'StartDate'})
df_Y = df_Y.drop(['Sector_Number', 'Exchange'], axis=1)

df_Y.head()
# df_Y.dtypes

,CompanyNumber,EventDate,EventType,StartDate,EventDate_string
0,15984000,2008-10-17,1,1999-03-15,2008 10
1,18729000,2023-07-31,0,2021-01-28,2023 07
2,22043000,2023-07-31,0,2015-08-26,2023 07
3,22043001,2013-04-30,2,2004-09-27,2013 04
4,26978000,2023-07-31,0,2009-05-05,2023 07


In [189]:
df_Y['StartDate'].isna()

0        False
1        False
2        False
3        False
4        False
         ...  
27383    False
27384    False
27385    False
27386    False
27387    False
Name: StartDate, Length: 27388, dtype: bool

In [242]:

for index, row in df_Y.iterrows():
    df_Y.at[index, 'CompanyNumber'] = row['CompanyNumber']/1000
df_Y['CompanyNumber'] = df_Y['CompanyNumber'].astype(int)

key_column = df_Y['CompanyNumber']
value_column = df_Y['StartDate']

# Create a dictionary from the two columns
comp_start_date = dict(zip(key_column, value_column))


C:\Users\shiho\AppData\Local\Temp\ipykernel_54368\759584361.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '22043.001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_Y.at[index, 'CompanyNumber'] = row['CompanyNumber']/1000


In [243]:
df_Y

,CompanyNumber,EventDate,EventType,StartDate,EventDate_string
0,15984,2008-10-17,1,1999-03-15,2008 10
1,18729,2023-07-31,0,2021-01-28,2023 07
2,22043,2023-07-31,0,2015-08-26,2023 07
3,22043,2013-04-30,2,2004-09-27,2013 04
4,26978,2023-07-31,0,2009-05-05,2023 07
...,...,...,...,...,...
27383,211986,2023-07-31,0,2023-06-14,2023 07
27384,212037,2023-07-31,0,2023-07-05,2023 07
27385,212083,2023-07-31,0,2023-06-30,2023 07
27386,212122,2023-07-31,0,2023-07-06,2023 07


In [244]:
comp_start_date


{15984: Timestamp('1999-03-15 00:00:00'),
 18729: Timestamp('2021-01-28 00:00:00'),
 22043: Timestamp('2004-09-27 00:00:00'),
 26978: Timestamp('1988-01-04 00:00:00'),
 26979: Timestamp('1988-01-04 00:00:00'),
 26980: Timestamp('1988-01-04 00:00:00'),
 26981: Timestamp('1988-01-04 00:00:00'),
 26982: Timestamp('1991-08-21 00:00:00'),
 26983: Timestamp('1988-01-04 00:00:00'),
 26984: Timestamp('2006-01-24 00:00:00'),
 26985: Timestamp('1988-01-04 00:00:00'),
 26986: Timestamp('1988-01-04 00:00:00'),
 26989: Timestamp('2005-09-01 00:00:00'),
 26990: Timestamp('1988-01-04 00:00:00'),
 26991: Timestamp('1994-10-21 00:00:00'),
 26992: Timestamp('1988-01-04 00:00:00'),
 26994: Timestamp('1988-01-04 00:00:00'),
 26995: Timestamp('1988-01-04 00:00:00'),
 26996: Timestamp('1988-01-04 00:00:00'),
 26998: Timestamp('1988-01-04 00:00:00'),
 26999: Timestamp('2006-10-12 00:00:00'),
 27000: Timestamp('1988-01-04 00:00:00'),
 27001: Timestamp('1988-01-04 00:00:00'),
 27002: Timestamp('1988-01-04 00:0

In [266]:
import pandas as pd
from datetime import datetime, timedelta

df_Y_new = pd.DataFrame()

# # Format the "EventDate" column to show year and month (yyyy mm)
# df_Y['EventDate'] = df_Y['EventDate'].dt.strftime('%Y %m')

# Sort the DataFrame by CompanyNumber and EventDate
df_Y.sort_values(by=['CompanyNumber', 'EventDate'], inplace=True)

# Function to check if event type 1 occurred n months ago for the same comp number
df_Y_new  # Declare df_Y_new as a global variable

for index, row in df_Y.iterrows():  
    comp_number = row['CompanyNumber']
    event_date = row['EventDate']

    if row['EventType'] == 1:
        for i in range(1, 13):  # Check for the past 12 months
            # Convert event_date to datetime format
            event_date = pd.to_datetime(event_date, format='%Y %m')
                # Calculate the date i months ago from the current event date
            i_months_ago = event_date - pd.DateOffset(months=i)

            # Check if the combination of CompanyNumber and i_months_ago is not in the original df_Y
            if not ((df_Y['CompanyNumber'] == comp_number) & (df_Y['EventDate_string'] == i_months_ago.strftime('%Y %m'))).any():
                # If not present, add a new row to df_Y_new
                new_row = {'EventDate_string': i_months_ago.strftime('%Y %m'), 'CompanyNumber': comp_number, 'EventType': 1, 'StartDate':row['StartDate']}
                new_df = pd.DataFrame([new_row])
                df_Y_new = pd.concat([df_Y_new, new_df], ignore_index=True)







# new_df

In [274]:
new_df = pd.concat([df_Y, df_Y_new], ignore_index=True)
new_df = new_df.drop_duplicates(subset=['EventDate_string', 'CompanyNumber','EventType'], keep='last')
new_df = new_df.drop(['EventDate','StartDate'], axis=1)

In [275]:
new_df

,CompanyNumber,EventType,EventDate_string
0,7998,2,2011 07
1,7998,2,2019 04
2,7998,0,2023 07
3,14547,2,2009 06
4,14547,2,2016 11
...,...,...,...
49745,209445,1,2022 06
49746,209445,1,2022 05
49747,209445,1,2022 04
49748,209445,1,2022 03


In [248]:
import numpy as np

df_X['Duration'] = np.nan

for index, row in df_X.iterrows():
    try:
        startDate = comp_start_date[row['CompanyNumber']]
        df_X.at[index, 'StartDate'] = startDate
        df_X.at[index, 'Duration'] = (row['EventDate'] - startDate)
    except KeyError:
        pass

C:\Users\shiho\AppData\Local\Temp\ipykernel_54368\2233856948.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-14 days +00:00:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_X.at[index, 'Duration'] = (row['EventDate'] - startDate)


In [250]:
df_X.head()

22203

In [251]:
comp_start_date 

{15984: Timestamp('1999-03-15 00:00:00'),
 18729: Timestamp('2021-01-28 00:00:00'),
 22043: Timestamp('2004-09-27 00:00:00'),
 26978: Timestamp('1988-01-04 00:00:00'),
 26979: Timestamp('1988-01-04 00:00:00'),
 26980: Timestamp('1988-01-04 00:00:00'),
 26981: Timestamp('1988-01-04 00:00:00'),
 26982: Timestamp('1991-08-21 00:00:00'),
 26983: Timestamp('1988-01-04 00:00:00'),
 26984: Timestamp('2006-01-24 00:00:00'),
 26985: Timestamp('1988-01-04 00:00:00'),
 26986: Timestamp('1988-01-04 00:00:00'),
 26989: Timestamp('2005-09-01 00:00:00'),
 26990: Timestamp('1988-01-04 00:00:00'),
 26991: Timestamp('1994-10-21 00:00:00'),
 26992: Timestamp('1988-01-04 00:00:00'),
 26994: Timestamp('1988-01-04 00:00:00'),
 26995: Timestamp('1988-01-04 00:00:00'),
 26996: Timestamp('1988-01-04 00:00:00'),
 26998: Timestamp('1988-01-04 00:00:00'),
 26999: Timestamp('2006-10-12 00:00:00'),
 27000: Timestamp('1988-01-04 00:00:00'),
 27001: Timestamp('1988-01-04 00:00:00'),
 27002: Timestamp('1988-01-04 00:0

In [271]:
df_X

,CompanyNumber,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,EventDate,EventDate_string,Duration,StartDate
0,15984,1999,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.324205,0,1999-03-01,1999 03,-14 days +00:00:00,1999-03-15
1,15984,1999,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.373554,0,1999-04-01,1999 04,17 days 00:00:00,1999-03-15
2,15984,1999,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,...,0.274280,0.0,0.0,0.000000,2.401488,0,1999-05-01,1999 05,47 days 00:00:00,1999-03-15
3,15984,1999,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,...,0.276896,0.0,0.0,0.000000,2.455890,0,1999-06-01,1999 06,78 days 00:00:00,1999-03-15
4,15984,1999,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,...,0.259755,0.0,0.0,0.000000,2.455596,0,1999-07-01,1999 07,108 days 00:00:00,1999-03-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753072,212037,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.087496,0,2023-07-01,2023 07,-4 days +00:00:00,2023-07-05
2753073,212083,2023,6,0.038825,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,NaN,NaN,NaN,3.405230,0.000000,1,2023-06-01,2023 06,-29 days +00:00:00,2023-06-30
2753074,212083,2023,7,0.015960,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,NaN,NaN,NaN,3.453191,0.000000,1,2023-07-01,2023 07,1 days 00:00:00,2023-06-30
2753075,212122,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.087496,0,2023-07-01,2023 07,-5 days +00:00:00,2023-07-06


In [280]:
from pandas_gbq import to_gbq
import json

result = pd.merge(df_X, new_df, on=['CompanyNumber', 'EventDate_string'], how='outer')
result['EventType'] = result['EventType'].fillna(0)


In [292]:
with open('../token.json', 'r') as token_file:
    token_data = json.load(token_file)
    project_id = token_data.get('project_id', 'default-project-id')

dataset_id = "capstone"
table_id = 'X_Y_merged_3'

to_gbq(result, destination_table=f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

In [293]:
result.isna().sum().sort_values(ascending=False)

dtdlevel            435285
dtdtrend            435285
liqnonfinlevel      333045
liqnonfintrend      333045
m2b                 323843
ni2talevel          315340
ni2tatrend          315340
sigma               309311
sizetrend           180306
sizelevel           180306
DTDmedianNonFin     146258
liqfinlevel          84390
lqfintrend           84390
StkIndx              66614
DTDmedianFin         49489
STInt                36889
yyyy                 19571
mm                   19571
dummy297fin          19571
EventDate            19571
StartDate            19571
Duration                 0
CompanyNumber            0
EventDate_string         0
EventType                0
dtype: int64

In [143]:
count_event_type_1 = len(result[result['EventType'] == 1.0])
count_event_type_1

16252

In [17]:
result.shape
result

,CompNo,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,EventDate,Event_type,Sector_Number,Exchange
0,15984,1999,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.324205,0,199903,NaN,NaN,NaN
1,15984,1999,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.373554,0,199904,NaN,NaN,NaN
2,15984,1999,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,...,0.274280,0.0,0.0,0.000000,2.401488,0,199905,NaN,NaN,NaN
3,15984,1999,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,...,0.276896,0.0,0.0,0.000000,2.455890,0,199906,NaN,NaN,NaN
4,15984,1999,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,...,0.259755,0.0,0.0,0.000000,2.455596,0,199907,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753075,212037,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.087496,0,202307,0.0,18.0,16.0
2753076,212083,2023,6,0.038825,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,NaN,NaN,NaN,3.405230,0.000000,1,202306,NaN,NaN,NaN
2753077,212083,2023,7,0.015960,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,NaN,NaN,NaN,3.453191,0.000000,1,202307,0.0,14.0,16.0
2753078,212122,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.087496,0,202307,0.0,12.0,16.0
